In [49]:
import pandas as pd

summary = []
summary.append({
    "GWAS": "GCST90450224", 
    "leads": "example_data/example_leads.tsv", 
    "GWAS_path": "example_data/GCST90450224.tsv",
    "chr_col": "chromosome",
    "pos_col": "base_pair_location",
    "maf_col": "effect_allele_frequency",
    "beta_col": "beta",
    "se_col": "standard_error",
    "file_type": "tsv",
    "ref_col": "other_allele",
    "alt_col": "effect_allele"
    })

pd.DataFrame(summary).to_csv("example_gwas_table.tsv",sep="\t", index=False)

In [ ]:
import pandas as pd

summary = []
summary.append({
    "id": "QTD000186", 
    "path": "example_data/QTD000186.lbf_variable.txt.gz.tsv",
    })

pd.DataFrame(summary).to_csv("example_eQTL_table.tsv",sep="\t", index=False)

In [48]:
import pandas as pd

df = pd.read_csv("GCST90450224.tsv", sep="\t")

df

,chromosome,base_pair_location,effect_allele,other_allele,beta,standard_error,effect_allele_frequency,neg_log_10_p_value,rsid,variant_id,info,ref_allele,n
0,1,285339,A,C,-0.020926,0.199508,0.014613,0.037884,rs879732627,1_285339_C_A,0.662782,OA,928
1,1,631712,C,T,0.161614,0.154556,0.017865,0.529126,rs9326622,1_631712_T_C,0.851058,OA,928
2,1,631739,C,A,0.161388,0.154579,0.017789,0.528031,rs9283153,1_631739_A_C,0.854172,OA,928
3,1,656627,A,G,-0.147961,0.125803,0.039830,0.620621,rs771664146,1_656627_G_A,0.661197,OA,928
4,1,656671,T,C,0.022934,0.175279,0.021045,0.047740,rs10218594,1_656671_C_T,0.609434,OA,928
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787942,1,248934255,A,C,0.131715,0.175541,0.013527,0.343852,rs138689967,1_248934255_C_A,0.953348,OA,928
787943,1,248934257,C,A,-0.114116,0.099312,0.046349,0.601138,rs199655021,1_248934257_A_C,0.903370,OA,928
787944,1,248936569,T,C,-0.047267,0.104786,0.038687,0.185798,rs532053890,1_248936569_C_T,0.877370,OA,928
787945,1,248936926,G,C,-0.088907,0.144758,0.019251,0.268333,rs199758136,1_248936926_C_G,0.948789,OA,928


In [37]:
# df = df[df["effect_allele_frequency"]>=0.05]

In [42]:
import numpy as np

def approx_bf_estimates(variant, z, V, type, suffix=None, sdY=1, effect_priors={'quant': 0.15, 'cc': 0.2}):
    sd_prior = effect_priors['quant'] * sdY if type == "quant" else effect_priors['cc']
    
    r = sd_prior**2 / (sd_prior**2 + V)
    lABF = 0.5 * (np.log(1 - r) + (r * z**2))
    
    ret = pd.DataFrame({"variant": variant, 'lbf': lABF})
    
    if suffix is not None:
        ret.columns = [f"{col}.{suffix}" for col in ret.columns]

    ret.loc[:, 'lbf'] = pd.to_numeric(ret['lbf'], errors='coerce')

    ret_mat = pd.DataFrame(ret.set_index('variant')['lbf']).T

    return ret_mat

df["variant"] = (
    "chr" + "1" + "_" +
    df["base_pair_location"].astype(int).astype(str) + "_" +
    df["other_allele"].astype(str) + "_" +
    df["effect_allele"].astype(str)
)

df["z"] = df["beta"] / df["standard_error"]
df["V"] = df["standard_error"] ** 2

mat = approx_bf_estimates(df["variant"],df["z"], df["V"], "quant") 
mat1 = mat.T
leads = mat1[mat1["lbf"]>=6].reset_index()
leads["position"] = leads["variant"].astype(str).str.split("_").str[1].astype(int)
leads

,variant,lbf,position
0,chr1_14826873_G_A,6.792095,14826873
1,chr1_76433270_G_A,6.541357,76433270
2,chr1_85516585_T_C,6.163146,85516585
3,chr1_85517720_A_G,6.057619,85517720
4,chr1_85543257_T_C,6.284835,85543257
5,chr1_218585370_G_A,6.246734,218585370
6,chr1_218588339_C_T,7.502817,218588339


In [43]:
leads["position"].unique()

array([ 14826873,  76433270,  85516585,  85517720,  85543257, 218585370,
       218588339])

In [45]:
leads_df= df[df["base_pair_location"].isin(leads["position"].unique())]
leads_df

leads_df.to_csv("example_leads.tsv", sep="\t")